In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import os
from google.colab import drive

# Montar Google Drive para almacenar los archivos extraídos
drive.mount('/content/drive')

def extraer_pelicula_filmaffinity(url_pelicula):
    """
    Extrae datos detallados de una película en FilmAffinity

    Args:
        url_pelicula (str): URL de la página de la película en FilmAffinity

    Returns:
        dict: Diccionario con todos los datos extraídos de la película, o None si hay errores
    """
    # Definir cabeceras HTTP para simular un navegador y evitar bloqueos
    cabeceras = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    print(f"Extrayendo datos de: {url_pelicula}")

    try:
        # Realizar petición HTTP a la página de la película
        respuesta = requests.get(url_pelicula, headers=cabeceras)

        # Verificar si la petición fue exitosa
        if respuesta.status_code != 200:
            print(f"Error al acceder a {url_pelicula}: Código {respuesta.status_code}")
            return None

        # Parsear el HTML de la página
        sopa = BeautifulSoup(respuesta.text, 'html.parser')

        # Inicializar diccionario para almacenar los datos de la película
        datos_pelicula = {}

        # Guardar URL de la película
        datos_pelicula['url'] = url_pelicula

        # ===== EXTRAER TÍTULO ORIGINAL =====
        titulo_original = None

        # Buscar explícitamente "Título original"
        elemento_titulo_original = sopa.select_one('dt:-soup-contains("Título original") + dd')
        if elemento_titulo_original:
            # Eliminar el span "show-akas" del elemento si existe
            span_aka = elemento_titulo_original.select_one('span.show-akas')
            if span_aka:
                span_aka.decompose()  # Elimina el span del árbol DOM

            # Extraer el texto limpio
            titulo_original = elemento_titulo_original.text.strip()

        datos_pelicula['titulo_original'] = titulo_original if titulo_original else "No disponible"

        # ===== EXTRAER AÑO =====
        anio = None

        # Buscar en atributo datePublished
        elemento_anio = sopa.select_one('dd[itemprop="datePublished"]')
        if elemento_anio:
            coincidencia_anio = re.search(r'\d{4}', elemento_anio.text.strip())
            if coincidencia_anio:
                anio = coincidencia_anio.group(0)

        datos_pelicula['anio'] = anio if anio else "No disponible"

        # ===== EXTRAER DURACIÓN =====
        duracion = None

        # Buscar con atributo duration
        elemento_duracion = sopa.select_one('dd[itemprop="duration"]')
        if elemento_duracion:
            duracion = elemento_duracion.text.strip()

        datos_pelicula['duracion'] = duracion if duracion else "No disponible"

        # ===== EXTRAER PAÍS =====
        pais = None

        # Buscar texto después de la bandera
        if not pais:
            imagen_pais = sopa.select_one('span[id="country-img"]')
            if imagen_pais and imagen_pais.next_sibling:
                texto_pais = imagen_pais.next_sibling.strip()
                if texto_pais:
                    pais = texto_pais

        datos_pelicula['pais'] = pais if pais else "No disponible"

        # ===== EXTRAER DIRECCIÓN =====
        direccion = None

        # Buscar en sección de créditos
        if not direccion:
            seccion_directores = sopa.select_one('dt:-soup-contains("Dirección") + dd')
            if seccion_directores:
                enlaces_directores = seccion_directores.select('a')
                if enlaces_directores:
                    direccion = ', '.join([enlace.text.strip() for enlace in enlaces_directores])
                else:
                    direccion = seccion_directores.text.strip()

        datos_pelicula['direccion'] = direccion if direccion else "No disponible"

        # ===== EXTRAER GUION =====
        guion = None

        # Buscar sección de guion
        seccion_guion = sopa.select_one('dt:-soup-contains("Guion") + dd')
        if seccion_guion:
            enlaces_guion = seccion_guion.select('a')
            if enlaces_guion:
                guion = ', '.join([enlace.text.strip() for enlace in enlaces_guion])
            else:
                guion = seccion_guion.text.strip()

        datos_pelicula['guion'] = guion if guion else "No disponible"

        # ===== EXTRAER REPARTO =====
        reparto = None

        # Extraer del carrusel de actores
        elementos_reparto = sopa.select('.credits-scroller .nb')
        if elementos_reparto:
            nombres_reparto = []
            for actor in elementos_reparto:
                nombre_actor = actor.select_one('a')
                if nombre_actor and not 'Ver todos los créditos' in nombre_actor.text:
                    nombres_reparto.append(nombre_actor.text.strip())
            if nombres_reparto:
                reparto = ', '.join(nombres_reparto)

        datos_pelicula['reparto'] = reparto if reparto else "No disponible"

        # ===== EXTRAER GÉNERO =====
        genero = None

        # Buscar género con itemprop
        elemento_genero = sopa.select_one('span[itemprop="genre"]')
        if elemento_genero:
            genero = elemento_genero.text.strip()

        datos_pelicula['genero'] = genero if genero else "No disponible"

        # ===== EXTRAER SINOPSIS =====
        sinopsis = None

        # Buscar sinopsis con itemprop
        elemento_sinopsis = sopa.select_one('dd[itemprop="description"]')
        if elemento_sinopsis:
            sinopsis = elemento_sinopsis.text.strip()

        datos_pelicula['sinopsis'] = sinopsis if sinopsis else "No disponible"

        # ===== EXTRAER URL DE LA IMAGEN =====
        url_imagen = None

        # Buscar en contenedor de imagen principal
        elemento_img = sopa.select_one('#movie-main-image-container img')
        if elemento_img and 'src' in elemento_img.attrs:
            url_imagen = elemento_img['src']
            # Intentar obtener imagen de mayor resolución
            url_imagen = url_imagen.replace('mmed', 'large')

        datos_pelicula['url_imagen'] = url_imagen if url_imagen else "No disponible"

        # ===== EXTRAER PUNTUACIÓN =====
        puntuacion = None

        # Buscar en elemento con ID movie-rat-avg
        elemento_valoracion = sopa.select_one('#movie-rat-avg')
        if elemento_valoracion:
            texto_valoracion = elemento_valoracion.text.strip()
            # Asegurarse de que solo contiene dígitos y posiblemente una coma
            if texto_valoracion:
                # Convertir coma a punto para formato decimal
                texto_valoracion = texto_valoracion.replace(',', '.')
                try:
                    puntuacion = float(texto_valoracion)
                except ValueError:
                    puntuacion = texto_valoracion

        datos_pelicula['puntuacion'] = puntuacion if puntuacion else "No disponible"

        # ===== EXTRAER NÚMERO DE VOTOS =====
        votos = None

        # Buscar en elemento con ID movie-count-rat span
        elemento_votos = sopa.select_one('#movie-count-rat span')
        if elemento_votos:
            texto_votos = elemento_votos.text.strip()
            # Limpiar puntos o espacios para obtener solo el número
            texto_votos = texto_votos.replace('.', '').replace(' ', '')
            try:
                votos = int(texto_votos)
            except ValueError:
                votos = texto_votos

        datos_pelicula['votos'] = votos if votos else "No disponible"

        print(f"Datos extraídos correctamente para {datos_pelicula['titulo_original']} ({datos_pelicula['anio']}) - Puntuación: {datos_pelicula['puntuacion']}")
        return datos_pelicula

    except Exception as e:
        print(f"Error al procesar {url_pelicula}: {str(e)}")
        return None

def obtener_peliculas_de_ranking_anual(url_ranking, max_peliculas=30):
    """
    Obtiene URLs de películas del ranking de un año específico

    Args:
        url_ranking (str): URL del ranking anual en FilmAffinity
        max_peliculas (int): Número máximo de películas a extraer

    Returns:
        list: Lista de URLs de películas encontradas en el ranking
    """
    # Definir cabeceras HTTP para simular un navegador
    cabeceras = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    urls_peliculas = []

    try:
        print(f"Accediendo al ranking: {url_ranking}")
        # Realizar petición HTTP al ranking
        respuesta = requests.get(url_ranking, headers=cabeceras)

        # Verificar si la petición fue exitosa
        if respuesta.status_code != 200:
            print(f"Error al acceder al ranking: Código {respuesta.status_code}")
            return []

        # Parsear el HTML de la página
        sopa = BeautifulSoup(respuesta.text, 'html.parser')

        # Buscar elementos de películas en la búsqueda avanzada (topgen.php)
        elementos_pelicula = sopa.select('.mc-right .movie-card')
        if not elementos_pelicula:
            # Si no encuentra con ese selector, probar con los selectores originales
            elementos_pelicula = sopa.select('.movie-card')

        print(f"Encontrados {len(elementos_pelicula)} elementos")

        # Procesar los elementos encontrados
        peliculas_encontradas = 0

        for elemento in elementos_pelicula:
            # Extraer el enlace de la película
            href = None

            if elemento.name == 'a' and '/film' in elemento.get('href', ''):
                href = elemento['href']
            elif elemento.name == 'div':
                enlace = elemento.select_one('a[href*="/film"]')
                if enlace:
                    href = enlace['href']
            else:
                # Intentar encontrar un enlace dentro del elemento
                enlace = elemento.select_one('a[href*="/film"]')
                if enlace:
                    href = enlace['href']

            if not href:
                continue

            # Construir URL completa
            url_pelicula = f"https://www.filmaffinity.com{href}" if href.startswith('/') else href

            if url_pelicula not in urls_peliculas:  # Evitar duplicados
                urls_peliculas.append(url_pelicula)
                peliculas_encontradas += 1
                print(f"Añadida película #{peliculas_encontradas} del ranking: {url_pelicula}")

                if peliculas_encontradas >= max_peliculas:
                    break

        print(f"Encontradas {peliculas_encontradas} películas en el ranking")

    except Exception as e:
        print(f"Error al procesar el ranking {url_ranking}: {str(e)}")

    return urls_peliculas

def exportar_a_excel(datos_peliculas, ruta_salida):
    """
    Exporta los datos de películas a un archivo Excel

    Args:
        datos_peliculas (list): Lista de diccionarios con datos de películas
        ruta_salida (str): Ruta donde se guardará el archivo Excel

    Returns:
        str: Ruta del archivo Excel generado, o None si hay error
    """
    # Verificar que hay datos
    if not datos_peliculas:
        print("No hay datos para exportar a Excel")
        return None

    # Crear DataFrame
    df = pd.DataFrame(datos_peliculas)

    # Reordenar las columnas para que aparezcan en el orden deseado
    orden_columnas = [
        'titulo_original', 'anio', 'duracion', 'pais', 'direccion',
        'guion', 'reparto', 'genero', 'sinopsis', 'url_imagen', 'puntuacion', 'votos', 'url'
    ]

    # Asegurarse de que solo se incluyen las columnas que existen
    columnas_validas = [col for col in orden_columnas if col in df.columns]
    df = df[columnas_validas]

    try:
        # Asegurarse de que el directorio existe
        directorio = os.path.dirname(ruta_salida)
        if directorio and not os.path.exists(directorio):
            print(f"Creando directorio: {directorio}")
            os.makedirs(directorio, exist_ok=True)

        # Exportar a Excel
        df.to_excel(ruta_salida, index=False)
        print(f"Datos exportados correctamente a {ruta_salida}")
        return ruta_salida
    except Exception as e:
        print(f"Error al exportar a Excel: {str(e)}")

def generar_url_ranking_topgen(anio):
    """
    Genera una URL para el buscador avanzado (topgen.php) de FilmAffinity para un año específico

    Args:
        anio (int): Año para el que se generará la URL

    Returns:
        str: URL completa para el ranking del año especificado
    """
    return f"https://www.filmaffinity.com/es/topgen.php?genres=&chv=0&orderby=rc&movietype=movie%7C&country=&fromyear={anio}&toyear={anio}&ratingcount=2&runtimemin=0&runtimemax=4"

def principal(anio_inicio, peliculas_por_anio=30, duracion_periodo=5):
    """
    Función principal que coordina la extracción completa de películas por año
    y guarda todos los resultados en un único archivo Excel

    Args:
        anio_inicio (int): Año desde el que se empezará a extraer
        peliculas_por_anio (int): Número de películas a extraer por cada año
        duracion_periodo (int): Duración del período en años (5 o 10)
    """
    # Generar el rango de años a partir del año de inicio
    anio_fin = anio_inicio + duracion_periodo - 1
    rango_anios = range(anio_inicio, anio_fin + 1)

    print(f"=== INICIANDO EXTRACCIÓN DE PELÍCULAS ==")
    print(f"Se extraerán hasta {peliculas_por_anio} películas por año ({anio_inicio}-{anio_fin})")
    print(f"Duración del período: {duracion_periodo} años")

    # Definir ruta para el archivo final
    ruta_archivo_final = f"/content/drive/My Drive/Tipología/PEC2/scraping/peliculas_filmaffinity_{anio_inicio}-{anio_fin}.xlsx"

    datos_todas_peliculas = []

    # Para cada año, extraer las películas
    for anio in rango_anios:
        print(f"\n=== Procesando año {anio} ===")

        # Generar URL para el año actual
        url_ranking = generar_url_ranking_topgen(anio)

        # Obtener URLs de las películas
        urls_peliculas = obtener_peliculas_de_ranking_anual(url_ranking, max_peliculas=peliculas_por_anio)

        if not urls_peliculas:
            print(f"No se encontraron películas para el año {anio}")
            continue

        # Extraer datos de cada película
        peliculas_procesadas = 0

        for i, url_pelicula in enumerate(urls_peliculas):
            print(f"\nProcesando película {i+1}/{len(urls_peliculas)} del año {anio}")

            # Extraer datos
            datos_pelicula = extraer_pelicula_filmaffinity(url_pelicula)

            if datos_pelicula:
                datos_todas_peliculas.append(datos_pelicula)
                peliculas_procesadas += 1
                print(f"Película {peliculas_procesadas} del año {anio} procesada correctamente")

            # Pausa para evitar sobrecarga del servidor
            tiempo_pausa = random.uniform(3, 7)
            print(f"Pausa de {tiempo_pausa:.2f} segundos para evitar sobrecarga...")
            time.sleep(tiempo_pausa)

        print(f"\nProcesadas {peliculas_procesadas} películas del año {anio}")

        # Pausa entre años
        tiempo_pausa = random.uniform(5, 10)
        print(f"Pausa de {tiempo_pausa:.2f} segundos antes de procesar el siguiente año...")
        time.sleep(tiempo_pausa)

    # Guardar todos los datos en un archivo Excel
    if datos_todas_peliculas:
        exportar_a_excel(datos_todas_peliculas, ruta_archivo_final)

        print("\n=== RESUMEN DE LA EXTRACCIÓN ===")
        print(f"Total de películas extraídas: {len(datos_todas_peliculas)}")
        print(f"Datos guardados en: {ruta_archivo_final}")

        # Mostrar distribución por año
        conteo_anios = {}
        for pelicula in datos_todas_peliculas:
            anio = pelicula['anio']
            if anio in conteo_anios:
                conteo_anios[anio] += 1
            else:
                conteo_anios[anio] = 1

        print("\nDistribución por año:")
        for anio, conteo in sorted(conteo_anios.items()):
            print(f"- {anio}: {conteo} películas")
    else:
        print("No se pudo extraer información de ninguna película")

def ejecutar_prueba(anio_inicio, duracion_periodo=5):
    """
    Función para ejecutar una prueba extrayendo una sola película por año

    Args:
        anio_inicio (int): Año desde el que se empezará a extraer
        duracion_periodo (int): Duración del período en años (5 o 10)
    """
    # Generar el rango de años a partir del año de inicio
    anio_fin = anio_inicio + duracion_periodo - 1
    rango_anios = range(anio_inicio, anio_fin + 1)

    print("=== INICIANDO PRUEBA DE EXTRACCIÓN ===")
    print(f"Se extraerá 1 película de cada año ({anio_inicio}-{anio_fin})")
    print(f"Duración del período: {duracion_periodo} años")

    # Usar la ruta específica dentro de Google Drive
    ruta_salida = f"/content/drive/My Drive/Tipología/PEC2/scraping/peliculas_filmaffinity_prueba_{anio_inicio}-{anio_fin}.xlsx"
    print(f"La salida se guardará en: {ruta_salida}")

    datos_todas_peliculas = []

    for anio in rango_anios:
        print(f"\n=== Procesando año {anio} ===")

        # Generar URL para el año actual
        url_ranking = generar_url_ranking_topgen(anio)

        # Obtener URL de la película
        urls_peliculas = obtener_peliculas_de_ranking_anual(url_ranking, max_peliculas=1)

        if urls_peliculas:
            # Extraer datos de la película
            datos_pelicula = extraer_pelicula_filmaffinity(urls_peliculas[0])

            if datos_pelicula:
                datos_todas_peliculas.append(datos_pelicula)
                print(f"Película del año {anio} extraída correctamente")
            else:
                print(f"No se pudo extraer la información de la película del año {anio}")
        else:
            print(f"No se encontraron películas para el año {anio}")

        # Pausa para evitar sobrecarga del servidor
        time.sleep(random.uniform(2, 4))

    # Exportar datos a Excel
    if datos_todas_peliculas:
        archivo_excel = exportar_a_excel(datos_todas_peliculas, ruta_salida)

        print("\n=== RESULTADOS DE LA PRUEBA ===")
        print(f"Se han extraído {len(datos_todas_peliculas)} películas")
        print(f"Los datos se han guardado en: {archivo_excel}")

        # Mostrar resumen de los datos
        print("\nResumen de las películas extraídas:")
        for pelicula in datos_todas_peliculas:
            print(f"- {pelicula['titulo_original']} ({pelicula['anio']})")
            print(f"  País: {pelicula['pais']}")
            print(f"  Duración: {pelicula['duracion']}")
            print(f"  Director: {pelicula['direccion']}")
            print(f"  Género: {pelicula['genero']}")
            print(f"  Puntuación: {pelicula['puntuacion']} ({pelicula['votos']} votos)")
    else:
        print("No se pudo extraer información de ninguna película")

def validar_anio(texto_anio, duracion_periodo):
    """
    Valida que el año introducido sea un número entero válido entre 1980 y el año actual

    Args:
        texto_anio (str): Texto ingresado por el usuario
        duracion_periodo (int): Duración del período en años (5 o 10)

    Returns:
        int or None: El año validado como entero o None si no es válido
    """
    try:
        anio = int(texto_anio)
        anio_actual = 2024  # Actualizar según sea necesario

        if anio < 1980 or anio > anio_actual - duracion_periodo + 1:  # Ajuste para período seleccionado
            print(f"El año debe estar entre 1980 y {anio_actual - duracion_periodo + 1} para un período de {duracion_periodo} años")
            return None

        return anio
    except ValueError:
        print("Por favor, introduce un número entero válido como año")
        return None

if __name__ == "__main__":
    # Menú de opciones
    print("=== SCRAPER DE FILMAFFINITY ===")
    print("1. Ejecutar prueba (1 película por año)")
    print("2. Ejecutar extracción completa (30 películas por año)")

    opcion = input("\nSelecciona una opción (1 o 2): ")

    # Solicitar duración del período
    duracion_periodo = None
    while duracion_periodo is None:
        try:
            texto_duracion = input("\n¿Cuántos años quieres abarcar? (5 o 10): ")
            duracion_periodo = int(texto_duracion)
            if duracion_periodo not in [5, 10]:
                print("Por favor, introduce 5 o 10 como duración del período")
                duracion_periodo = None
        except ValueError:
            print("Por favor, introduce un número entero (5 o 10)")
            duracion_periodo = None

    # Solicitar año de inicio (desde 1980 en adelante)
    anio_valido = None
    while anio_valido is None:
        texto_anio = input("\nIntroduce el año de inicio (desde 1980): ")
        anio_valido = validar_anio(texto_anio, duracion_periodo)

    if opcion == "1":
        ejecutar_prueba(anio_valido, duracion_periodo)
    elif opcion == "2":
        # Preguntar cuántas películas por año
        try:
            num_peliculas = int(input("\n¿Cuántas películas quieres extraer por año? (por defecto: 30): ") or "30")
            if num_peliculas <= 0:
                raise ValueError("El número debe ser positivo")
            principal(anio_valido, peliculas_por_anio=num_peliculas, duracion_periodo=duracion_periodo)
        except ValueError as e:
            print(f"Error: {e}")
            print("Se usará el valor por defecto (30 películas).")
            principal(anio_valido, peliculas_por_anio=30, duracion_periodo=duracion_periodo)
    else:
        print("Opción no válida. Por favor, selecciona 1 o 2.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== SCRAPER DE FILMAFFINITY ===
1. Ejecutar prueba (1 película por año)
2. Ejecutar extracción completa (30 películas por año)

Selecciona una opción (1 o 2): 2

¿Cuántos años quieres abarcar? (5 o 10): 10

Introduce el año de inicio (desde 1980): 1990

¿Cuántas películas quieres extraer por año? (por defecto: 30): 4
=== INICIANDO EXTRACCIÓN DE PELÍCULAS ==
Se extraerán hasta 4 películas por año (1990-1999)
Duración del período: 10 años

=== Procesando año 1990 ===
Accediendo al ranking: https://www.filmaffinity.com/es/topgen.php?genres=&chv=0&orderby=rc&movietype=movie%7C&country=&fromyear=1990&toyear=1990&ratingcount=2&runtimemin=0&runtimemax=4
Encontrados 30 elementos
Añadida película #1 del ranking: https://www.filmaffinity.com/es/film514728.html
Añadida película #2 del ranking: https://www.filmaffinity.com/es/film783311.html
Añadida película #3 del ranki